## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Feb. 23rd Mar. 2nd


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,February-27-2025,Condo/Co-op,4647 Kittiwake Ct Unit Kingfisher,Boynton Beach,FL,33436.0,325000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boynton-Beach/4647-K...,Beaches MLS,RX-11066378,N,Y,26.509302,-80.111706


In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [8]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [9]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [10]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-02-28    42
2025-02-27    28
2025-02-25    24
2025-02-24    21
2025-02-26    14
Name: count, dtype: int64

In [11]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [12]:
df_top_ten = df2.head(10)

In [13]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
35,PAST SALE,2025-02-24,Condo/Co-op,2700 N Ocean Dr NE Unit Ts7a,Singer Island,FL,33404.0,11280000.0,5.0,5.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11048624,N,Y,26.785105,-80.034110
28,PAST SALE,2025-02-28,Condo/Co-op,2 N Breakers Row Unit N21,Palm Beach,FL,33480.0,7300000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-11034900,N,Y,26.717146,-80.034396
8,PAST SALE,2025-02-26,Condo/Co-op,20155 Boca West Dr Unit B-302,Boca Raton,FL,33434.0,3500000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-W-B...,Beaches MLS,RX-11053204,N,Y,26.377428,-80.161759
73,PAST SALE,2025-02-27,Condo/Co-op,200 Arkona Ct #1902,West Palm Beach,FL,33401.0,3250000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/200-...,Beaches MLS,RX-11034962,N,Y,26.699821,-80.051397
101,PAST SALE,2025-02-25,Condo/Co-op,3740 S Ocean Blvd #1504,Highland Beach,FL,33487.0,2600000.0,3.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3740-...,Beaches MLS,RX-11014431,N,Y,26.407547,-80.065582
13,PAST SALE,2025-02-27,Condo/Co-op,200 Macfarlane Dr Unit N-901,Delray Beach,FL,33483.0,2195000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Beaches MLS,F10483873,N,Y,26.457711,-80.063636
12,PAST SALE,2025-02-28,Condo/Co-op,550 S Ocean Blvd #1506,Boca Raton,FL,33432.0,1850000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11059080,N,Y,26.344431,-80.070692
42,PAST SALE,2025-02-24,Condo/Co-op,2667 N Ocean Blvd Unit 602-I,Boca Raton,FL,33431.0,1800000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2667-N-Oc...,Beaches MLS,RX-11039332,N,Y,26.374737,-80.068854
3,PAST SALE,2025-02-27,Condo/Co-op,550 Okeechobee Blvd Unit Mph-02,West Palm Beach,FL,33401.0,1475000.0,3.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/550-...,Beaches MLS,RX-11061259,N,Y,26.705124,-80.056199
124,PAST SALE,2025-02-26,Condo/Co-op,3100 N Ocean Dr Unit 1406 P,Singer Island,FL,33404.0,1400000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Beaches MLS,RX-10971433,N,Y,26.788221,-80.034608


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [14]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [15]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [16]:
response_df = pd.DataFrame(response_list)

In [17]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [18]:
df_top_ten = merged_df

In [19]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-02-24,Condo/Co-op,2700 N Ocean Dr NE Unit Ts7a,Singer Island,FL,33404.0,11280000.0,5.0,5.5,...,-80.034110,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Deborah Strand,Douglas Elliman,None,None,Amy Simmonds PA,Compass Florida LLC,None,None
1,PAST SALE,2025-02-28,Condo/Co-op,2 N Breakers Row Unit N21,Palm Beach,FL,33480.0,7300000.0,3.0,3.5,...,-80.034396,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,James McCann,"Premier Estate Properties, Inc",David Scaff,"Premier Estate Properties, Inc",Carole Koeppel,"Sotheby's Intl. Realty, Inc.",None,None
2,PAST SALE,2025-02-26,Condo/Co-op,20155 Boca West Dr Unit B-302,Boca Raton,FL,33434.0,3500000.0,3.0,3.5,...,-80.161759,https://www.redfin.com/FL/Boca-Raton/20155-W-B...,Rebecca Spooner,Siemens Group Realty Corp.,Michael Spooner,Siemens Group Realty Corp.,Janice Gates,Boca West Realty LLC,None,None
3,PAST SALE,2025-02-27,Condo/Co-op,200 Arkona Ct #1902,West Palm Beach,FL,33401.0,3250000.0,2.0,2.5,...,-80.051397,https://www.redfin.com/FL/West-Palm-Beach/200-...,Joseph Columbo,Douglas Elliman,Christopher Leavitt,Douglas Elliman,Kirsten Smith,Douglas Elliman,None,None
4,PAST SALE,2025-02-25,Condo/Co-op,3740 S Ocean Blvd #1504,Highland Beach,FL,33487.0,2600000.0,3.0,2.5,...,-80.065582,https://www.redfin.com/FL/Highland-Beach/3740-...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Janet Sivert,BHHS EWM Realty,None,None
5,PAST SALE,2025-02-27,Condo/Co-op,200 Macfarlane Dr Unit N-901,Delray Beach,FL,33483.0,2195000.0,3.0,2.0,...,-80.063636,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Allison Stewart,Compass Florida LLC,None,None,Karen Stone,Keller Williams Realty Boca Raton,None,None
6,PAST SALE,2025-02-28,Condo/Co-op,550 S Ocean Blvd #1506,Boca Raton,FL,33432.0,1850000.0,2.0,2.0,...,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None
7,PAST SALE,2025-02-24,Condo/Co-op,2667 N Ocean Blvd Unit 602-I,Boca Raton,FL,33431.0,1800000.0,3.0,3.0,...,-80.068854,https://www.redfin.com/FL/Boca-Raton/2667-N-Oc...,Ronald Posner,Century 21 Stein Posner,None,None,Jonathan Phillips,Keller Williams Realty Services,None,None
8,PAST SALE,2025-02-27,Condo/Co-op,550 Okeechobee Blvd Unit Mph-02,West Palm Beach,FL,33401.0,1475000.0,3.0,2.0,...,-80.056199,https://www.redfin.com/FL/West-Palm-Beach/550-...,Lilia Caballero,Premier Brokers International,None,None,Lilia Caballero,Premier Brokers International,None,None
9,PAST SALE,2025-02-26,Condo/Co-op,3100 N Ocean Dr Unit 1406 P,Singer Island,FL,33404.0,1400000.0,2.0,2.0,...,-80.034608,https://www.redfin.com/FL/Riviera-Beach/3100-N...,Angela Voland,Waterfront Properties & Club C,None,None,Non Agent,Non-Member Selling Office,None,None


## Current Week's Values

In [20]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

129


In [21]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$617,735


In [22]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$366


In [23]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$79,687,754


In [25]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PDF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PDF  Last Week Total Dollar Volume
0          129              617735          366                     79687754.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [26]:
print('Input county name:')
county = input()
print('Input Previous Week Condo Sales Total (number units sold):')
previous_week_condo_sales = input()
print('Input Previous Week Condo Average Sales Price:')
previous_week_average_sales_price = input()
print('Input Previous Week Condo Average PSF:')
previous_week_average_psf = input()
print('Input Previous Week Condo Sales total (ex: 198_000_000)')
previous_week_condo_sales_total = input()

## Clean Data

Input county name:
Palm Beach County
Input Previous Week Condo Sales Total (number units sold):
68
Input Previous Week Condo Average Sales Price:
1036772
Input Previous Week Condo Average PSF:
497
Input Previous Week Condo Sales total (ex: 198_000_000)
70_500_000


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [27]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [28]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [29]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR'],
      dtype='object')

In [30]:
pd.set_option('display.max_columns',None)

In [31]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-24,Condo/Co-op,2700 N Ocean Dr NE Unit Ts7a,Singer Island,FL,33404.0,11280000.0,5.0,5.5,2700 North Ocean Condo,7445.0,NaN,2008.0,NaN,1515.0,8826.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11048624,N,Y,26.785105,-80.03411,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Deborah Strand,Douglas Elliman,None,None,Amy Simmonds PA,Compass Florida LLC,None,None,1,orange


In [32]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [33]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-24,Condo/Co-op,2700 N Ocean Dr NE Unit Ts7a,Singer Island,FL,33404.0,11280000.0,5.0,5.5,2700 North Ocean Condo,7445.0,NaN,2008.0,NaN,1515.0,8826.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11048624,N,Y,26.785105,-80.034110,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Deborah Strand,Douglas Elliman,None,None,Amy Simmonds PA,Compass Florida LLC,None,None,1,orange
1,PAST SALE,2025-02-28,Condo/Co-op,2 N Breakers Row Unit N21,Palm Beach,FL,33480.0,7300000.0,3.0,3.5,Two North Breakers Row Condo,3474.0,NaN,1986.0,NaN,2101.0,16562.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-11034900,N,Y,26.717146,-80.034396,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,James McCann,"Premier Estate Properties, Inc",David Scaff,"Premier Estate Properties, Inc",Carole Koeppel,"Sotheby's Intl. Realty, Inc.",None,None,2,blue
2,PAST SALE,2025-02-26,Condo/Co-op,20155 Boca West Dr Unit B-302,Boca Raton,FL,33434.0,3500000.0,3.0,3.5,Akoya Boca West,2277.0,NaN,2019.0,NaN,1537.0,4152.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-W-B...,Beaches MLS,RX-11053204,N,Y,26.377428,-80.161759,https://www.redfin.com/FL/Boca-Raton/20155-W-B...,Rebecca Spooner,Siemens Group Realty Corp.,Michael Spooner,Siemens Group Realty Corp.,Janice Gates,Boca West Realty LLC,None,None,3,blue
3,PAST SALE,2025-02-27,Condo/Co-op,200 Arkona Ct #1902,West Palm Beach,FL,33401.0,3250000.0,2.0,2.5,LA Clara,1516.0,NaN,2023.0,NaN,2144.0,2620.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/200-...,Beaches MLS,RX-11034962,N,Y,26.699821,-80.051397,https://www.redfin.com/FL/West-Palm-Beach/200-...,Joseph Columbo,Douglas Elliman,Christopher Leavitt,Douglas Elliman,Kirsten Smith,Douglas Elliman,None,None,4,blue
4,PAST SALE,2025-02-25,Condo/Co-op,3740 S Ocean Blvd #1504,Highland Beach,FL,33487.0,2600000.0,3.0,2.5,Toscana South Condo,2601.0,NaN,2004.0,NaN,1000.0,2422.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3740-...,Beaches MLS,RX-11014431,N,Y,26.407547,-80.065582,https://www.redfin.com/FL/Highland-Beach/3740-...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Janet Sivert,BHHS EWM Realty,None,None,5,blue
5,PAST SALE,2025-02-27,Condo/Co-op,200 Macfarlane Dr Unit N-901,Delray Beach,FL,33483.0,2195000.0,3.0,2.0,Seagate Towers Condo,1760.0,NaN,1970.0,NaN,1247.0,1839.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Beaches MLS,F10483873,N,Y,26.457711,-80.063636,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Allison Stewart,Compass Florida LLC,None,None,Karen Stone,Keller Williams Realty Boca Raton,None,None,6,blue
6,PAST SALE,2025-02-28,Condo/Co-op,550 S Ocean Blvd #1506,Boca Raton,FL,33432.0,1850000.0,2.0,2.0,Chalfonte Condo,1448.0,NaN,1974.0,NaN,1278.0,1525.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11059080,N,Y,26.344431,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None,7,blue
7,PAST SALE,2025-02-24,Condo/Co-op,2667 N Ocean Blvd Unit 602-I,Boca Raton,FL,33431.0,1800000.0,3.0,3.0,Yacht & Racquet Club OF Boca Raton Condo,1871.0,NaN,1980.0,NaN,962.0,2376.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/2667-N-Oc...,Beaches MLS,RX-11039332,N,Y,26.374737,-80.068854,https://www.redfin.com/FL/Boca-Raton/2667-N-Oc...,Ronald Posner,Century 21 Stein Posner,None,None,Jonathan Phillips,Keller Williams Realty Services,None,None,8,blue
8,PAST SALE,2025-02-27,Cond

In [34]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [35]:
m.save('index.html')

## Data snagger

In [36]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [37]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR
0,PAST SALE,2025-02-24,Condo/Co-op,2700 N Ocean Dr NE Unit Ts7a,Singer Island,FL,33404.0,11280000.0,5.0,5.5,2700 North Ocean Condo,7445.0,NaN,2008.0,NaN,1515.0,8826.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11048624,N,Y,26.785105,-80.03411,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Deborah Strand,Douglas Elliman,None,None,Amy Simmonds PA,Compass Florida LLC,None,None,1,orange


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [38]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_03032024


# CREATE TEMPLATE 

In [39]:
muni_set = set(df_top_ten['CITY'])

In [40]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [41]:
df_top_ten.reset_index(inplace=True,drop=True)

In [42]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [43]:
top_sale

'2700 North Ocean Condo at 2700 N Ocean Dr NE Unit Ts7a in Singer Island'

In [44]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [45]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [46]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-24,Condo/Co-op,2700 N Ocean Dr NE Unit Ts7a,Singer Island,FL,33404.0,11280000.0,5.0,5.5,2700 North Ocean Condo,7445.0,NaN,2008.0,NaN,1515.0,8826.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11048624,N,Y,26.785105,-80.034110,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Deborah Strand,Douglas Elliman,None,None,Amy Simmonds PA,Compass Florida LLC,None,None,1,orange,2700 North Ocean Condo at 2700 N Ocean Dr NE U...
1,PAST SALE,2025-02-28,Condo/Co-op,2 N Breakers Row Unit N21,Palm Beach,FL,33480.0,7300000.0,3.0,3.5,Two North Breakers Row Condo,3474.0,NaN,1986.0,NaN,2101.0,16562.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-11034900,N,Y,26.717146,-80.034396,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,James McCann,"Premier Estate Properties, Inc",David Scaff,"Premier Estate Properties, Inc",Carole Koeppel,"Sotheby's Intl. Realty, Inc.",None,None,2,blue,Two North Breakers Row Condo at 2 N Breakers R...
2,PAST SALE,2025-02-26,Condo/Co-op,20155 Boca West Dr Unit B-302,Boca Raton,FL,33434.0,3500000.0,3.0,3.5,Akoya Boca West,2277.0,NaN,2019.0,NaN,1537.0,4152.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-W-B...,Beaches MLS,RX-11053204,N,Y,26.377428,-80.161759,https://www.redfin.com/FL/Boca-Raton/20155-W-B...,Rebecca Spooner,Siemens Group Realty Corp.,Michael Spooner,Siemens Group Realty Corp.,Janice Gates,Boca West Realty LLC,None,None,3,blue,Akoya Boca West at 20155 Boca West Dr Unit B-3...
3,PAST SALE,2025-02-27,Condo/Co-op,200 Arkona Ct #1902,West Palm Beach,FL,33401.0,3250000.0,2.0,2.5,LA Clara,1516.0,NaN,2023.0,NaN,2144.0,2620.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/200-...,Beaches MLS,RX-11034962,N,Y,26.699821,-80.051397,https://www.redfin.com/FL/West-Palm-Beach/200-...,Joseph Columbo,Douglas Elliman,Christopher Leavitt,Douglas Elliman,Kirsten Smith,Douglas Elliman,None,None,4,blue,LA Clara at 200 Arkona Ct #1902 in West Palm B...
4,PAST SALE,2025-02-25,Condo/Co-op,3740 S Ocean Blvd #1504,Highland Beach,FL,33487.0,2600000.0,3.0,2.5,Toscana South Condo,2601.0,NaN,2004.0,NaN,1000.0,2422.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3740-...,Beaches MLS,RX-11014431,N,Y,26.407547,-80.065582,https://www.redfin.com/FL/Highland-Beach/3740-...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Janet Sivert,BHHS EWM Realty,None,None,5,blue,Toscana South Condo at 3740 S Ocean Blvd #1504...
5,PAST SALE,2025-02-27,Condo/Co-op,200 Macfarlane Dr Unit N-901,Delray Beach,FL,33483.0,2195000.0,3.0,2.0,Seagate Towers Condo,1760.0,NaN,1970.0,NaN,1247.0,1839.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Beaches MLS,F10483873,N,Y,26.457711,-80.063636,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Allison Stewart,Compass Florida LLC,None,None,Karen Stone,Keller Williams Realty Boca Raton,None,None,6,blue,Seagate Towers Condo at 200 Macfarlane Dr Unit...
6,PAST SALE,2025-02-28,Condo/Co-op,550 S Ocean Blvd #1506,Boca Raton,FL,33432.0,1850000.0,2.0,2.0,Chalfonte Condo,1448.0,NaN,1974.0,NaN,1278.0,1525.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11059080,N,Y,26.344431,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None,7,blue,Chalfonte Condo at 550 S Ocean Blvd #1506 in B...
7,PAST SALE,2025-02-24,Condo/Co-op,2667 N Ocean Blvd Unit 602-I,Boca Raton,FL,33431.0,1800000.0,3.0,3.0,Yacht & Racquet Club

In [47]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: 2700 North Ocean Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $1,400,000 to $11,280,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Delray Beach, West Palm Beach, Singer Island, Palm Beach, Highland Beach, Boca Raton, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 129 condo sales totaling $79,687,754 million from Feb. 23rd to Mar. 2nd. The previous week, brokers closed 68 condo sales totaling $70,500,000.

Last week’s units sol

In [48]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [49]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [50]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [51]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-24,Condo/Co-op,2700 N Ocean Dr NE Unit Ts7a,Singer Island,FL,33404.0,11280000.0,5.0,5.5,2700 North Ocean Condo,7445.0,NaN,2008.0,NaN,1515.0,8826.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11048624,N,Y,26.785105,-80.034110,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Deborah Strand,Douglas Elliman,None,None,Amy Simmonds PA,Compass Florida LLC,None,None,1,orange,2700 North Ocean Condo at 2700 N Ocean Dr NE U...
1,PAST SALE,2025-02-28,Condo/Co-op,2 N Breakers Row Unit N21,Palm Beach,FL,33480.0,7300000.0,3.0,3.5,Two North Breakers Row Condo,3474.0,NaN,1986.0,NaN,2101.0,16562.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-11034900,N,Y,26.717146,-80.034396,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,James McCann,"Premier Estate Properties, Inc",David Scaff,"Premier Estate Properties, Inc",Carole Koeppel,"Sotheby's Intl. Realty, Inc.",None,None,2,blue,Two North Breakers Row Condo at 2 N Breakers R...
2,PAST SALE,2025-02-26,Condo/Co-op,20155 Boca West Dr Unit B-302,Boca Raton,FL,33434.0,3500000.0,3.0,3.5,Akoya Boca West,2277.0,NaN,2019.0,NaN,1537.0,4152.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-W-B...,Beaches MLS,RX-11053204,N,Y,26.377428,-80.161759,https://www.redfin.com/FL/Boca-Raton/20155-W-B...,Rebecca Spooner,Siemens Group Realty Corp.,Michael Spooner,Siemens Group Realty Corp.,Janice Gates,Boca West Realty LLC,None,None,3,blue,Akoya Boca West at 20155 Boca West Dr Unit B-3...
3,PAST SALE,2025-02-27,Condo/Co-op,200 Arkona Ct #1902,West Palm Beach,FL,33401.0,3250000.0,2.0,2.5,LA Clara,1516.0,NaN,2023.0,NaN,2144.0,2620.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/200-...,Beaches MLS,RX-11034962,N,Y,26.699821,-80.051397,https://www.redfin.com/FL/West-Palm-Beach/200-...,Joseph Columbo,Douglas Elliman,Christopher Leavitt,Douglas Elliman,Kirsten Smith,Douglas Elliman,None,None,4,blue,LA Clara at 200 Arkona Ct #1902 in West Palm B...
4,PAST SALE,2025-02-25,Condo/Co-op,3740 S Ocean Blvd #1504,Highland Beach,FL,33487.0,2600000.0,3.0,2.5,Toscana South Condo,2601.0,NaN,2004.0,NaN,1000.0,2422.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3740-...,Beaches MLS,RX-11014431,N,Y,26.407547,-80.065582,https://www.redfin.com/FL/Highland-Beach/3740-...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Janet Sivert,BHHS EWM Realty,None,None,5,blue,Toscana South Condo at 3740 S Ocean Blvd #1504...
5,PAST SALE,2025-02-27,Condo/Co-op,200 Macfarlane Dr Unit N-901,Delray Beach,FL,33483.0,2195000.0,3.0,2.0,Seagate Towers Condo,1760.0,NaN,1970.0,NaN,1247.0,1839.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Beaches MLS,F10483873,N,Y,26.457711,-80.063636,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Allison Stewart,Compass Florida LLC,None,None,Karen Stone,Keller Williams Realty Boca Raton,None,None,6,blue,Seagate Towers Condo at 200 Macfarlane Dr Unit...
6,PAST SALE,2025-02-28,Condo/Co-op,550 S Ocean Blvd #1506,Boca Raton,FL,33432.0,1850000.0,2.0,2.0,Chalfonte Condo,1448.0,NaN,1974.0,NaN,1278.0,1525.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11059080,N,Y,26.344431,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None,7,blue,Chalfonte Condo at 550 S Ocean Blvd #1506 in B...
7,PAST SALE,2025-02-24,Condo/Co-op,2667 N Ocean Blvd Unit 602-I,Boca Raton,FL,33431.0,1800000.0,3.0,3.0,Yacht & Racquet Club

In [52]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: 2700 North Ocean Condo closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $1,400,000 to $11,280,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Delray Beach, West Palm Beach, Singer Island, Palm Beach, Highland Beach, Boca Raton, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 129 condo sales totaling $79,687,754 million from Feb. 23rd to Mar. 2nd. The previous week, brokers closed 68 condo sales totaling $70,500,000.

Last week’s units sol

In [66]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Riviera-Beach/2700-N-Ocean-Dr-33404/unit-TS-7A/home/78367394


In [67]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Palm-Beach/2-N-Breakers-Row-33480/unit-N21/home/42435124


In [62]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Riviera-Beach/3100-N-Ocean-Dr-33404/unit-1406/home/187721241


In [63]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/West-Palm-Beach/550-Okeechobee-Blvd-33401/unit-2/home/42663907


In [64]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/West-Palm-Beach/200-Arkona-Ct-33401/unit-1902/home/186263378


In [58]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Riviera-Beach/3100-N-Ocean-Dr-33404/unit-1406/home/187721241


In [59]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-02-24,Condo/Co-op,2700 N Ocean Dr NE Unit Ts7a,Singer Island,FL,33404.0,11280000.0,5.0,5.5,2700 North Ocean Condo,7445.0,NaN,2008.0,NaN,1515.0,8826.0,Sold,NaN,NaN,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Beaches MLS,RX-11048624,N,Y,26.785105,-80.034110,https://www.redfin.com/FL/Riviera-Beach/2700-N...,Deborah Strand,Douglas Elliman,None,None,Amy Simmonds PA,Compass Florida LLC,None,None,1,orange,2700 North Ocean Condo at 2700 N Ocean Dr NE U...
1,PAST SALE,2025-02-28,Condo/Co-op,2 N Breakers Row Unit N21,Palm Beach,FL,33480.0,7300000.0,3.0,3.5,Two North Breakers Row Condo,3474.0,NaN,1986.0,NaN,2101.0,16562.0,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,Beaches MLS,RX-11034900,N,Y,26.717146,-80.034396,https://www.redfin.com/FL/Palm-Beach/2-N-Break...,James McCann,"Premier Estate Properties, Inc",David Scaff,"Premier Estate Properties, Inc",Carole Koeppel,"Sotheby's Intl. Realty, Inc.",None,None,2,blue,Two North Breakers Row Condo at 2 N Breakers R...
2,PAST SALE,2025-02-26,Condo/Co-op,20155 Boca West Dr Unit B-302,Boca Raton,FL,33434.0,3500000.0,3.0,3.5,Akoya Boca West,2277.0,NaN,2019.0,NaN,1537.0,4152.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-W-B...,Beaches MLS,RX-11053204,N,Y,26.377428,-80.161759,https://www.redfin.com/FL/Boca-Raton/20155-W-B...,Rebecca Spooner,Siemens Group Realty Corp.,Michael Spooner,Siemens Group Realty Corp.,Janice Gates,Boca West Realty LLC,None,None,3,blue,Akoya Boca West at 20155 Boca West Dr Unit B-3...
3,PAST SALE,2025-02-27,Condo/Co-op,200 Arkona Ct #1902,West Palm Beach,FL,33401.0,3250000.0,2.0,2.5,LA Clara,1516.0,NaN,2023.0,NaN,2144.0,2620.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/200-...,Beaches MLS,RX-11034962,N,Y,26.699821,-80.051397,https://www.redfin.com/FL/West-Palm-Beach/200-...,Joseph Columbo,Douglas Elliman,Christopher Leavitt,Douglas Elliman,Kirsten Smith,Douglas Elliman,None,None,4,blue,LA Clara at 200 Arkona Ct #1902 in West Palm B...
4,PAST SALE,2025-02-25,Condo/Co-op,3740 S Ocean Blvd #1504,Highland Beach,FL,33487.0,2600000.0,3.0,2.5,Toscana South Condo,2601.0,NaN,2004.0,NaN,1000.0,2422.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3740-...,Beaches MLS,RX-11014431,N,Y,26.407547,-80.065582,https://www.redfin.com/FL/Highland-Beach/3740-...,Michael Wells,Premier Estate Properties Inc,Scot Karp,Premier Estate Properties Inc,Janet Sivert,BHHS EWM Realty,None,None,5,blue,Toscana South Condo at 3740 S Ocean Blvd #1504...
5,PAST SALE,2025-02-27,Condo/Co-op,200 Macfarlane Dr Unit N-901,Delray Beach,FL,33483.0,2195000.0,3.0,2.0,Seagate Towers Condo,1760.0,NaN,1970.0,NaN,1247.0,1839.0,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Beaches MLS,F10483873,N,Y,26.457711,-80.063636,https://www.redfin.com/FL/Delray-Beach/200-Mac...,Allison Stewart,Compass Florida LLC,None,None,Karen Stone,Keller Williams Realty Boca Raton,None,None,6,blue,Seagate Towers Condo at 200 Macfarlane Dr Unit...
6,PAST SALE,2025-02-28,Condo/Co-op,550 S Ocean Blvd #1506,Boca Raton,FL,33432.0,1850000.0,2.0,2.0,Chalfonte Condo,1448.0,NaN,1974.0,NaN,1278.0,1525.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Beaches MLS,RX-11059080,N,Y,26.344431,-80.070692,https://www.redfin.com/FL/Boca-Raton/550-S-Oce...,Ana Londono,Lang Realty/BR,None,None,Ana Londono,Lang Realty/BR,None,None,7,blue,Chalfonte Condo at 550 S Ocean Blvd #1506 in B...
7,PAST SALE,2025-02-24,Condo/Co-op,2667 N Ocean Blvd Unit 602-I,Boca Raton,FL,33431.0,1800000.0,3.0,3.0,Yacht & Racquet Club

## Time on Market Calculator

In [68]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 11, 7) ## List (Earlier) date
date2 = datetime(2025, 2, 28) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

113


## Clean CSV for Datawrapper Chart

In [69]:
chart_df = df_top_ten

In [70]:
chart_df = chart_df.fillna(" ")

In [71]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [72]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [73]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [74]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [75]:
chart_df['$/SQUARE FEET'].astype(float)

0    1515.0
1    2101.0
2    1537.0
3    2144.0
4    1000.0
5    1247.0
6    1278.0
7     962.0
8     918.0
9     870.0
Name: $/SQUARE FEET, dtype: float64

In [76]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [77]:
csv_date_string = today.strftime("%m_%d_%Y")

In [78]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [79]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-02-24,2700 N Ocean Dr NE Unit Ts7a in Singer Island,"$11,280,000",5,5.5,"7,445",2008,"$1,515",Beaches MLS,RX-11048624,Deborah Strand Douglas Elliman,Amy Simmonds PA Compass Florida LLC
1,2025-02-28,2 N Breakers Row Unit N21 in Palm Beach,"$7,300,000",3,3.5,"3,474",1986,"$2,101",Beaches MLS,RX-11034900,"James McCann Premier Estate Properties, Inc Da...","Carole Koeppel Sotheby's Intl. Realty, Inc."
2,2025-02-26,20155 Boca West Dr Unit B-302 in Boca Raton,"$3,500,000",3,3.5,"2,277",2019,"$1,537",Beaches MLS,RX-11053204,Rebecca Spooner Siemens Group Realty Corp. Mic...,Janice Gates Boca West Realty LLC
3,2025-02-27,200 Arkona Ct #1902 in West Palm Beach,"$3,250,000",2,2.5,"1,516",2023,"$2,144",Beaches MLS,RX-11034962,Joseph Columbo Douglas Elliman Christopher Lea...,Kirsten Smith Douglas Elliman
4,2025-02-25,3740 S Ocean Blvd #1504 in Highland Beach,"$2,600,000",3,2.5,"2,601",2004,"$1,000",Beaches MLS,RX-11014431,Michael Wells Premier Estate Properties Inc Sc...,Janet Sivert BHHS EWM Realty
5,2025-02-27,200 Macfarlane Dr Unit N-901 in Delray Beach,"$2,195,000",3,2.0,"1,760",1970,"$1,247",Beaches MLS,F10483873,Allison Stewart Compass Florida LLC,Karen Stone Keller Williams Realty Boca Raton
6,2025-02-28,550 S Ocean Blvd #1506 in Boca Raton,"$1,850,000",2,2.0,"1,448",1974,"$1,278",Beaches MLS,RX-11059080,Ana Londono Lang Realty/BR,Ana Londono Lang Realty/BR
7,2025-02-24,2667 N Ocean Blvd Unit 602-I in Boca Raton,"$1,800,000",3,3.0,"1,871",1980,$962,Beaches MLS,RX-11039332,Ronald Posner Century 21 Stein Posner,Jonathan Phillips Keller Williams Realty Servi...
8,2025-02-27,550 Okeechobee Blvd Unit Mph-02 in West Palm B...,"$1,475,000",3,2.0,"1,606",2008,$918,Beaches MLS,RX-11061259,Lilia Caballero Premier Brokers International,Lilia Caballero Premier Brokers International
9,2025-02-26,3100 N Ocean Dr Unit 1406 P in Singer Island,"$1,400,000",2,2.0,"1,610",2023,$870,Beaches MLS,RX-10971433,Angela Voland Waterfront Properties & Club C,Non Agent Non-Member Selling Office
